

Постановка задачи:

    Объединить в одну выборку
    Провести исследование и выявить тематики о которых говорят в твитах
    Сделать визуализацию кластеров тематик
    Проинтерпритировать получившиеся тематики



In [2]:
# Загрузка данных
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-20 07:15:02--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-20 07:15:03--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbea9130d93c53abea6d8c334e6.dl.dropboxusercontent.com/cd/0/inline/BnhEKTIhZLzgIoIOluMeo_UiwdmH3hNWCqdkgLffgtFNL32KN-C-ibwE7zIPoDaRik21yj7Jh6qNgCsUDoT315zc3gteG5XvvU3xseUx2pEG_7Q9z5PDMMuw3JRHPp6S4nkRaEBTGFz4_fzq0ZUrWQz9otqEb14_Yz7uvYbU288pmQ/file# [following]
--2022-06-20 07:15:03--  https://ucbea9130d93c53abea6d8c334e6.dl.dropboxusercontent.com/cd/0/inline/BnhEKTIhZLzgIoIOluMeo_UiwdmH3hNWCqdkgLffgtFNL32KN-C-ibwE7zIPoDaRi

In [1]:
# Установка доп библиотек
!pip install pymorphy2
!pip install pyLDAvis
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 8.2 MB 23.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 31.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=e6854d6be96453649671c719efe559aeb4de8075abd4ff02c9f5b01ab70f1b32
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-no

In [3]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
import string
from pymorphy2 import MorphAnalyzer
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
#Загружаем и объединяем датасеты
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['tweet'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['tweet'])

df = positive.append(negative)
df.sample(2)

,tweet
106439,"Никогда не обманывай людей, которые верят тебе..."
9948,"@yoursadness да) напиши ему кстати, а то мой в..."


Предобрабатываем датасет

In [5]:
morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
noise = set(sw + punkt)

def lemmatize_text(text):
    return [t for t in [morpher.parse(token)[0].normal_form 
                                for token in word_tokenize(text)] if t not in noise and t.isalpha() and len(t) >= 4]

In [6]:
# очистка токенов от ненужных для определения темы и нецензурных слов

def clear_tokens(tokens):
    token_str = ' '.join(tokens)
    token_str = re.sub(r'[A-za-z]', ' ',  token_str) 
    token_str = re.sub(r'\w+ще\b', ' ',  token_str) 
    token_str = re.sub(r'\bбл\w+', ' ',  token_str) 
    token_str = re.sub(r'\bидио\w+', ' ',  token_str) 
    token_str = re.sub(r'\bпид\w+', ' ',  token_str)
    token_str = re.sub(r'\bху\w+', ' ', token_str)
    token_str = re.sub(r'\w+xу.\b', ' ',  token_str)
    token_str = re.sub(r'\w+xу\w+', ' ',  token_str)
    token_str = re.sub(r'\w+хо\b', ' ',  token_str)
    token_str = re.sub(r'\bсук\w+', ' ',  token_str)
    token_str = re.sub(r'\bпоп\w+', ' ',  token_str)
    token_str = re.sub(r'\bпиз\w+', ' ',  token_str)
    token_str = re.sub(r'\bгов\w+', ' ',  token_str)
    token_str = re.sub(r'\bфиг\w+', ' ',  token_str)
    token_str = re.sub(r'\bпи.де\w+', ' ',  token_str)
    token_str = re.sub(r'никто',  ' ', token_str) 
    token_str = re.sub(r'завтра',  ' ', token_str) 
    token_str = re.sub(r'свой', ' ', token_str) 
    token_str = re.sub(r'\w+ть\b', ' ',  token_str) 
    token_str = re.sub(r'\w+ться\b', ' ',  token_str)
    token_str = re.sub(r'который', ' ', token_str)
    token_str = re.sub(r'нету', ' ', token_str)
    token_str = re.sub(r'вчера', ' ', token_str)
    token_str = re.sub(r'\w+ти\b', ' ', token_str)
    token_str = re.sub(r'самый', ' ', token_str)
    clear_tokens = token_str.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize_text(text)))

In [7]:
df['tweet'] = df['tweet'].apply(preprocess)
df.sample(5)

,tweet
64364,[народ]
49492,"[девочка, последний, обострение]"
20524,"[любимый, тимберлейк, загрузка]"
70400,"[ленивый, новогодний, маникюр, жалко, времча, ..."
79498,"[пятница, платье]"


Модель LDA для определения тематики

In [8]:
dictionary = corpora.Dictionary(df['tweet'])
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('tweet.dict')

In [9]:
#Векторизация
corpus = [dictionary.doc2bow(tweet) for tweet in df['tweet']]
corpora.MmCorpus.serialize('tweets.model', corpus) 

In [10]:
#Посмотрим на примере
print(df['tweet'].iloc[1])
print(corpus[1])

['похожий', 'мальчик', 'равно', 'хороший']
[(4, 1), (5, 1), (6, 1), (7, 1)]


In [41]:
#Обучаем модель
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=4)

In [42]:
lda.show_topics(num_topics=4, num_words=25, formatted=True)

[(0,
  '0.033*"хороший" + 0.021*"друг" + 0.019*"настроение" + 0.015*"плохой" + 0.014*"телефон" + 0.012*"снег" + 0.012*"скучно" + 0.011*"сильно" + 0.010*"твиттер" + 0.010*"слово" + 0.010*"вечер" + 0.010*"новый" + 0.010*"учёба" + 0.010*"большой" + 0.009*"боль" + 0.009*"месяц" + 0.009*"твит" + 0.009*"любимый" + 0.008*"больно" + 0.008*"погода" + 0.008*"нормально" + 0.007*"песня" + 0.007*"зима" + 0.007*"слеза" + 0.007*"история"'),
 (1,
  '0.032*"школа" + 0.021*"большой" + 0.020*"мама" + 0.016*"урок" + 0.015*"голова" + 0.013*"дело" + 0.013*"экзамен" + 0.012*"каникулы" + 0.011*"жалко" + 0.010*"новый" + 0.009*"нужный" + 0.009*"ранний" + 0.008*"капец" + 0.008*"момент" + 0.008*"должный" + 0.007*"папа" + 0.007*"наверное" + 0.007*"рука" + 0.007*"проблема" + 0.007*"сразу" + 0.007*"душа" + 0.006*"быстро" + 0.006*"родитель" + 0.006*"мысль" + 0.006*"черта"'),
 (2,
  '0.023*"утро" + 0.017*"ночь" + 0.013*"фильм" + 0.013*"прийтись" + 0.012*"ужасно" + 0.011*"правда" + 0.010*"ужасный" + 0.010*"город" + 0.0

In [43]:
#Оценка модели
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.00029564319768207893


In [44]:
coherence_model_lda = CoherenceModel(model=lda, texts=df['tweet'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

Средняя когерентность:  0.2489888690214019


В целом, модель показала неплохой результат с низкой персплексией и достаточно высокой когерентностью.

In [45]:
#Визуализация результатов модели
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

В качестве эксперимента модель была обучена на выделение 3, 5, 6, 7 и 10 тематик. При 6, 7 и 10 тематиках значения начали пересекаться. При 5 тематиках круги лежат слишком плотно друг к другу. Наилучший результат показало значение 4.

Выделенные тематики:
1. школа, учеба
2. работа
3. настроение, эмоции
4. досуг